In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

data = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

# Imputation
data.Age.fillna(data.Age.median(), inplace=True)
data.Fare.fillna(data.Fare.median(), inplace=True)
data.Embarked.fillna('Z', inplace=True)
test.Age.fillna(data.Age.median(), inplace=True)
test.Fare.fillna(data.Fare.median(), inplace=True)
test.Embarked.fillna('Z', inplace=True)

# Get index variables
dfx = data[['Age', 'Sex', 'Pclass', 'Parch', 'SibSp', 'Fare', 'Embarked']]
dfy = data[['Survived']]
dict_sex = {'male':0, 'female':1}
dict_emb = {'S':-0.75, 'Q':-0.25, 'C':0.25, 'Z':0.75}
dfx['Embarked'].replace(dict_emb, inplace=True)
dfx['Sex'].replace(dict_sex, inplace=True)
dfx['Fare'] = np.log(dfx['Fare'])
dtfx = test[['Age', 'Sex', 'Pclass', 'Parch', 'SibSp', 'Fare', 'Embarked']]
dtfx['Embarked'].replace(dict_emb, inplace=True)
dtfx['Sex'].replace(dict_sex, inplace=True)
dtfx['Fare'] = np.log(dtfx['Fare'])
feature_names = list(dfx.columns)

X = dfx.values.reshape(-1,len(feature_names))
y = dfy.values.ravel()
X_test = dtfx.values.reshape(-1, len(feature_names))

# # Regularization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = scaler.transform(X_test)

# Take it to the data
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB = NB.fit(X_scaled,y)
guess = NB.predict(X_test_scaled)
submit = [guess, test['PassengerId']]
# outputArray = []
df = pd.DataFrame(submit, index = ['PassengerId', 'Survived'])
df.to_csv(path_or_buf='submission.csv', index=False)